# 1. Import packages

In [1]:
import aiohttp,nest_asyncio
from google.cloud import bigquery
from utils_hack import get_eeat_prompts, run_assessment,prompt_api
from EEAT_calculator import score_eeat
import pandas as pd
import json

# 2. Set up google cloud and visualize databases

In [2]:
from google.cloud import bigquery

# Connect to BigQuery
PROJECT_ID = "dbce-cmap-seo-dev-9e60"
DATASET_ID = "hcktn_f_ai_search_insights"

client = bigquery.Client(project=PROJECT_ID)
print(f"Connected to project: {PROJECT_ID}")
print(f"Dataset: {DATASET_ID}")

/home/kassounians/projects/hackathon/.venv/lib/python3.12/site-packages/google/auth/_default.py:114: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Connected to project: dbce-cmap-seo-dev-9e60
Dataset: hcktn_f_ai_search_insights


In [3]:
# List tables in the dataset
dataset_ref = client.dataset(DATASET_ID)
tables = list(client.list_tables(dataset_ref))

print(f"Tables in {DATASET_ID}:")
for table in tables:
    print(f"  - {table.table_id}")

Tables in hcktn_f_ai_search_insights:
  - ai_readiness_score_timeline
  - ai_readiness_score_timeline_copy
  - all_checkpoints_storage
  - api_markdowns
  - crawlability_issues_checkpoint
  - crawlability_issues_checkpoint_backup_11_07
  - cwv_checkpoint
  - domain_authority
  - domain_authority_checkpoint
  - eeat_score_timeline
  - eeat_score_timeline_copy
  - image_alt_tags_checkpoint
  - image_alt_tags_checkpoint_copy2
  - js_issue_checkpoint
  - structured_data_checkpoint
  - url_prompts
  - url_status
  - urls_page_types
  - urls_page_types_api
  - urls_page_types_api_copy_backup
  - urls_page_types_copy_backup
  - urls_prompt_checkpoints_ref
  - urls_prompt_checkpoints_results
  - urls_prompt_results
  - urls_prompt_results_api
  - urls_prompt_results_backup_11_07


# 3. Initiate analysis for the data

* The hackathon organizer gave us a set of prepared prompts that we can use to analyze and generate an assesment for each input.
* It is recommended to use the prompts which have a "_m" in as they analyze websites and general text
* If the output is suspicious we can fix the parsers in the 

In [18]:
prompts = ['author_info_m',
 'contributor_author',
 'content_angle_m',
 'trust_m',
 'expertise_m',
 'content_quality_m',
 'experience_m',
 'ymyl']

In [19]:
df_prompt = pd.json_normalize(get_eeat_prompts()['data']).query('promptName in @prompts')

/home/kassounians/projects/hackathon/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.pgcloud.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [20]:
df_prompt.shape

(8, 9)

In [22]:
df_prompt

,promptName,tags,promptText,variables,temperature,model,responseModality,additionalParams,type
0,author_info_m,[EEAT],# Task\n Find information about article Author...,[{markdown}],0.7,GPT-41-MINI-2025-04-14,text,[],single
1,contributor_author,[EEAT],# Task\nFind information about the article's R...,[{URL}],0.7,GPT-4O-MINI-2024-07-18,text,[],single
2,content_angle_m,[EEAT],# Task\nEvaluate an article's URL provided as ...,[{Markdown}],0.7,GPT-41-MINI-2025-04-14,text,[],single
3,trust_m,[EEAT],# Task\n Audit the Content in terms of Trus...,[{Markdown}],0.7,GPT-4O-2024-11-20,text,[],single
4,expertise_m,[EEAT],# Task\n Audit the content in terms of Exp...,[{Markdown}],0.7,GPT-4O-MINI-2024-07-18,text,[],single
5,content_quality_m,[EEAT],# Task\n Audit the content in terms of ...,[{markdown}],0.7,GPT-4O-MINI-2024-07-18,text,[],single
8,experience_m,[EEAT],# Task\n Audit the Content in terms of Expe...,[{Markdown}],0.8,GEMINI-2.0-FLASH-LITE-001,text,[],single
12,ymyl,[EEAT],# Task\n Assess if Topic is YMYL or not b...,[{content}],0.7,GPT-4O-MINI-2024-07-18,text,[],single


In [41]:
sample_test =  { 'content':"""Hi my name is sarkis and I will take to you about my life. This is a link to my CV www.sarkisprowebsite.com, whereas this link talks about my personal life
                                     www.sarkisinsider.com.I'm from Lebanon and I have lived there for most of my life years. I studied there and I did my masters in Astrophysics. Then I went to armenia to do
                                     data science internships. Then I came to poland.
                                     """,
                 'markdown':"""Hi my name is sarkis and I will take to you about my life. This is a link to my CV www.sarkisprowebsite.com, whereas this link talks about my personal life
                                     www.sarkisinsider.com.I'm from Lebanon and I have lived there for most of my life years. I studied there and I did my masters in Astrophysics. Then I went to armenia to do
                                     data science internships. Then I came to poland.
                                     """,
                'url': "https://garagetested.com/how-to-use-power-tools/",

                
                'title': "Life of Sarkis",

                'html code': """
                            <!DOCTYPE html>
                            <html lang="en">
                            <head>
                                <meta charset="UTF-8">
                                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                                <title>About Sarkis</title>
                                <style>
                                    body {
                                        font-family: Arial, sans-serif;
                                        line-height: 1.6;
                                        margin: 20px;
                                    }
                                    h1 {
                                        color: #333;
                                    }
                                    p {
                                        margin: 10px 0;
                                    }
                                    a {
                                        color: #007BFF;
                                        text-decoration: none;
                                    }
                                    a:hover {
                                        text-decoration: underline;
                                    }
                                </style>
                            </head>
                            <body>
                                <h1>About Me</h1>
                                <p>Hi, my name is <strong>Sarkis</strong> and I will talk to you about my life.</p>
                                
                                <p>This is a link to my <a href="http://www.sarkisprowebsite.com" target="_blank">CV</a>, whereas this link talks about my personal life: <a href="http://www.sarkisinsider.com" target="_blank">Personal Life</a>.</p>
                                
                                <p>I was born in <strong>Syria</strong> and I grew up in <strong>Lebanon</strong> and I have <strong>Armenian origins</strong>. I studied there and completed my master's degree in <strong>Astrophysics</strong>.</p>
                                
                                <p>Then I went to <strong>Armenia</strong> to do data science internships. After that, I came to <strong>Poland</strong> for more data science opportunities.</p>
                            </body>
                            </html>""" 

                } 

# 4. Multi-thread and iterate

In [42]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def _build_args(row):
    """Prepare the keyword arguments for a single run_assessment call."""
    input_type = row.variables[0].replace("{", "").replace("}", "")
    input_title = sample_test["title"] if len(row.variables) == 2 else ""
    return {
        "input_value": sample_test["url"],
        "input_type": input_type,
        "prompt_type": row.promptName,
        "title": input_title,
    }

# Build work items
work_items = {row.promptName: _build_args(row) for row in df_prompt.itertuples()}

# Run all 22 API calls in parallel
total_res = {}
with ThreadPoolExecutor(max_workers=len(work_items)) as executor:
    futures = {
        executor.submit(run_assessment, **kwargs): prompt_name
        for prompt_name, kwargs in work_items.items()
    }
    for future in as_completed(futures):
        prompt_name = futures[future]
        try:
            total_res[prompt_name] = future.result()
        except Exception as e:
            print(f"[ERROR]: {prompt_name} failed – {e}")

print(f"[DONE]: {len(total_res)}/{len(work_items)} prompts completed")

/home/kassounians/projects/hackathon/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.pgcloud.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/kassounians/projects/hackathon/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.pgcloud.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/kassounians/projects/hackathon/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.pgcloud.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#t

[INFO]: LLM API response: contributor_author -> 200
[INFO]: LLM API response: author_info_m -> 200
[INFO]: LLM API response: content_angle_m -> 200
[INFO]: LLM API response: ymyl -> 200
[ERROR]: ymyl failed – Expecting value: line 3 column 16 (char 208)
[INFO]: LLM API response: content_quality_m -> 200
[INFO]: LLM API response: experience_m -> 200
[INFO]: LLM API response: expertise_m -> 200
[INFO]: LLM API response: trust_m -> 200
[ERROR]: trust_m failed – Extra data: line 30 column 1 (char 1358)
[DONE]: 6/8 prompts completed


# 5. Estimate score

In [55]:
data = {}
for prom in prompts:
    data[prom] = total_res[prom]['output']

In [56]:
scores = score_eeat(data)
print(scores)
# If you want a plain dict:
print({
    "E": scores.experience,
    "Ex": scores.expertise,
    "A": scores.authoritativeness,
    "T": scores.trust,
    "EEAT": scores.overall,
    "weights": scores.weights,
    "flags": scores.flags,
})

EEATScores(experience=100.0, expertise=67.11, authoritativeness=50.0, trust=70.0, overall=69.78, weights={'E': 0.15, 'Ex': 0.25, 'A': 0.2, 'T': 0.4}, flags=['angle_state:experience_led', 'missing_author', 'missing_reviewer_editor', 'missing_citations', 'ymyl_citation_cap_applied', 'ymyl_transparency_cap_applied'], angle_state='experience_led', is_clear_ymyl=True)
{'E': 100.0, 'Ex': 67.11, 'A': 50.0, 'T': 70.0, 'EEAT': 69.78, 'weights': {'E': 0.15, 'Ex': 0.25, 'A': 0.2, 'T': 0.4}, 'flags': ['angle_state:experience_led', 'missing_author', 'missing_reviewer_editor', 'missing_citations', 'ymyl_citation_cap_applied', 'ymyl_transparency_cap_applied']}
